# 3-Class Dog Emotion Recognition - Test & Visualization Notebook

## Key Corrections Made:

### 1. **Branch Configuration**
- Changed from `conf-merge-3cls` to `conf-3cls` (your actual branch)
- Repository: `https://github.com/hoangh-e/dog-emotion-recognition-hybrid.git`

### 2. **3-Class System**
- Classes: `['angry', 'happy', 'relaxed']` (NOT merged sad)
- Direct mapping: 0=angry, 1=happy, 2=relaxed
- No class merging needed (already 3-class from start)

### 3. **Model Loading Fixes**
- Proper paths for your model files
- Correct architecture parameters
- Fixed import statements

### 4. **YOLO Handling**
- YOLO trained on 3-class directly
- No conversion needed if YOLO outputs match

In [ ]:
# Download models
!gdown 1kg_O6D1i243veRSK2IDTxSqLFJ8Rie8l -O /content/vit.pt
!gdown 1i4Y0IldGspmHXNJv2Ypi0td6Knfg5ep3 -O /content/EfficientNet.pt
!gdown 1chEvbJzodR6Ifg9vQ-tDXzeLH0kXlmnD -O /content/densenet.pth
!gdown 1Io77ALDwVmZYwUtKDlxJ0m02J73aAUTA -O /content/alex.pth
!gdown 1Io77ALDwVmZYwUtKDlxJ0m02J73aAUTA -O /content/resnet101.pth
!gdown 1oP4XLqDxJmzhP5ztiD3VVvGr7I-6yT0P -O /content/yolo_11.pt

In [ ]:
import os, sys

REPO_URL = "https://github.com/hoangh-e/dog-emotion-recognition-hybrid.git"
BRANCH_NAME = "conf-3cls"  # CORRECTED: Use conf-3cls, not conf-merge-3cls
REPO_NAME = "dog-emotion-recognition-hybrid"

if not os.path.exists(REPO_NAME):
    !git clone -b $BRANCH_NAME $REPO_URL
    
os.chdir(REPO_NAME)
if os.getcwd() not in sys.path: 
    sys.path.insert(0, os.getcwd())

# Install dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install opencv-python-headless pillow pandas tqdm gdown albumentations 
!pip install matplotlib seaborn plotly scikit-learn timm ultralytics roboflow

In [ ]:
import numpy as np
import pandas as pd
import cv2
import torch
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from ultralytics import YOLO

# CORRECTED: 3-class configuration (no merging needed)
EMOTION_CLASSES = ['angry', 'happy', 'relaxed']  # Direct 3-class
NUM_CLASSES = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"✅ Configured for 3-class system: {EMOTION_CLASSES}")
print(f"🔧 Using device: {device}")

In [ ]:
# Import modules
from dog_emotion_classification import alexnet, densenet, efficientnet, vit, resnet

print("✅ Modules imported successfully")

# Define algorithms dictionary with correct parameters
ALGORITHMS = {
    'AlexNet': {
        'module': alexnet,
        'load_func': 'load_alexnet_model',
        'predict_func': 'predict_emotion_alexnet',
        'params': {'input_size': 224, 'num_classes': 3},
        'model_path': '/content/alex.pth'
    },
    'DenseNet121': {
        'module': densenet,
        'load_func': 'load_densenet_model',
        'predict_func': 'predict_emotion_densenet',
        'params': {'architecture': 'densenet121', 'input_size': 224, 'num_classes': 3},
        'model_path': '/content/densenet.pth'
    },
    'EfficientNet-B0': {
        'module': efficientnet,
        'load_func': 'load_efficientnet_model',
        'predict_func': 'predict_emotion_efficientnet',
        'params': {'architecture': 'efficientnet_b0', 'input_size': 224, 'num_classes': 3},
        'model_path': '/content/EfficientNet.pt'
    },
    'ViT': {
        'module': vit,
        'load_func': 'load_vit_model',
        'predict_func': 'predict_emotion_vit',
        'params': {'architecture': 'vit_b_16', 'input_size': 224, 'num_classes': 3},
        'model_path': '/content/vit.pt'
    },
    'ResNet101': {
        'module': resnet,
        'load_func': 'load_resnet_model',
        'predict_func': 'predict_emotion_resnet',
        'params': {'architecture': 'resnet101', 'input_size': 224, 'num_classes': 3},
        'model_path': '/content/resnet101.pth'
    }
}

In [ ]:
from roboflow import Roboflow
from pathlib import Path

# Download dataset
rf = Roboflow(api_key="blm6FIqi33eLS0ewVlKV")
project = rf.workspace("2642025").project("19-06")
version = project.version(7)
dataset = version.download("yolov12")

dataset_path = Path(dataset.location)
test_images_path = dataset_path / "test" / "images"
test_labels_path = dataset_path / "test" / "labels"
cropped_images_path = dataset_path / "cropped_test_images"
cropped_images_path.mkdir(exist_ok=True)

def crop_and_save_heads(image_path, label_path, output_dir):
    """Crop head regions - NO CLASS CONVERSION NEEDED (already 3-class)"""
    img = cv2.imread(str(image_path))
    if img is None: 
        return []
    
    h, w, _ = img.shape
    cropped_files = []
    
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
            
        for idx, line in enumerate(lines):
            cls, x, y, bw, bh = map(float, line.strip().split())
            
            # NO CONVERSION - already 3-class (0=angry, 1=happy, 2=relaxed)
            cls = int(cls)
            
            # Crop bounding box
            x1 = int((x - bw/2) * w)
            y1 = int((y - bh/2) * h)
            x2 = int((x + bw/2) * w)
            y2 = int((y + bh/2) * h)
            
            # Ensure within bounds
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)
            
            if x2 > x1 and y2 > y1:
                crop = img[y1:y2, x1:x2]
                crop_filename = output_dir / f"{image_path.stem}_{idx}_cls{cls}.jpg"
                cv2.imwrite(str(crop_filename), crop)
                
                cropped_files.append({
                    'filename': crop_filename.name,
                    'path': str(crop_filename),
                    'original_image': image_path.name,
                    'ground_truth': cls,
                    'bbox': [x1, y1, x2, y2]
                })
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
    
    return cropped_files

# Process all test images
all_cropped_data = []
for img_path in test_images_path.glob("*.jpg"):
    label_path = test_labels_path / (img_path.stem + ".txt")
    if label_path.exists():
        all_cropped_data.extend(crop_and_save_heads(img_path, label_path, cropped_images_path))

all_data_df = pd.DataFrame(all_cropped_data)

# Validate labels are 3-class
print(f"✅ Label distribution (should be 0, 1, 2):")
print(all_data_df['ground_truth'].value_counts().sort_index())

# Split into train/test
train_df, test_df = train_test_split(
    all_data_df, 
    test_size=0.2, 
    stratify=all_data_df['ground_truth'], 
    random_state=42
)

print(f"Train: {len(train_df)}, Test: {len(test_df)}")

In [ ]:
def load_yolo_emotion_model():
    try:
        model = YOLO('/content/yolo_11.pt')
        print("✅ YOLO model loaded")
        
        # Check YOLO classes
        if hasattr(model, 'names'):
            print(f"YOLO classes: {model.names}")
        
        return model
    except Exception as e:
        print(f"❌ Failed to load YOLO: {e}")
        return None

def predict_emotion_yolo(image_path, model, head_bbox=None, device='cuda'):
    try:
        results = model(image_path)
        if len(results) == 0 or len(results[0].boxes.cls) == 0:
            return {'predicted': False}
        
        cls_id = int(results[0].boxes.cls[0].item())
        conf = float(results[0].boxes.conf[0].item())
        
        # Direct mapping (no conversion needed if YOLO trained on 3-class)
        emotion_scores = {e: 0.0 for e in EMOTION_CLASSES}
        if 0 <= cls_id < len(EMOTION_CLASSES):
            emotion_scores[EMOTION_CLASSES[cls_id]] = conf
        else:
            return {'predicted': False}
            
        emotion_scores['predicted'] = True
        return emotion_scores
        
    except Exception as e:
        print(f"YOLO prediction error: {e}")
        return {'predicted': False}

# Load YOLO
yolo_emotion_model = load_yolo_emotion_model()

if yolo_emotion_model:
    ALGORITHMS['YOLO_Emotion'] = {
        'module': None,
        'custom_model': yolo_emotion_model,
        'custom_predict': predict_emotion_yolo
    }

In [ ]:
def load_standard_model(module, load_func_name, params, model_path, device='cuda'):
    """Load model with proper parameters"""
    import os
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found: {model_path}")
    
    load_func = getattr(module, load_func_name)
    
    # Handle different parameter formats
    if 'architecture' in params:
        result = load_func(
            model_path=model_path,
            architecture=params['architecture'],
            num_classes=params['num_classes'],
            input_size=params.get('input_size', 224),
            device=device
        )
    else:
        result = load_func(
            model_path=model_path,
            num_classes=params['num_classes'],
            input_size=params.get('input_size', 224),
            device=device
        )
    
    return result

# Load all models
loaded_models = {}

for name, config in ALGORITHMS.items():
    try:
        if 'custom_model' in config:
            # YOLO special case
            loaded_models[name] = {
                'model': config['custom_model'],
                'transform': None,
                'config': config
            }
            print(f"✅ {name} loaded")
        else:
            # Standard models
            result = load_standard_model(
                config['module'],
                config['load_func'],
                config['params'],
                config['model_path'],
                device
            )
            
            if isinstance(result, tuple):
                model, transform = result
            else:
                model = result
                transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                       std=[0.229, 0.224, 0.225])
                ])
            
            loaded_models[name] = {
                'model': model,
                'transform': transform,
                'config': config
            }
            print(f"✅ {name} loaded")
            
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")

print(f"\n✅ Loaded {len(loaded_models)}/{len(ALGORITHMS)} models")

In [ ]:
def test_algorithm_on_dataset(algorithm_name, model_data, df, max_samples=9999):
    """Test algorithm on dataset"""
    model = model_data['model']
    transform = model_data['transform']
    config = model_data['config']
    
    results = {
        'algorithm': algorithm_name,
        'predictions': [],
        'ground_truths': [],
        'confidences': [],
        'success_count': 0,
        'error_count': 0
    }
    
    for idx, row in df.head(max_samples).iterrows():
        try:
            if 'custom_predict' in config:
                # YOLO
                pred = config['custom_predict'](row['path'], model, device=device)
            else:
                # Standard models
                predict_func = getattr(config['module'], config['predict_func'])
                pred = predict_func(
                    image_path=row['path'],
                    model=model,
                    transform=transform,
                    device=device,
                    emotion_classes=EMOTION_CLASSES
                )
            
            if pred and pred.get('predicted', False):
                scores = {k: v for k, v in pred.items() if k != 'predicted'}
                pred_emotion = max(scores, key=scores.get)
                pred_class = EMOTION_CLASSES.index(pred_emotion)
                conf = scores[pred_emotion]
                
                results['predictions'].append(pred_class)
                results['ground_truths'].append(row['ground_truth'])
                results['confidences'].append(conf)
                results['success_count'] += 1
            else:
                results['error_count'] += 1
                
        except Exception as e:
            print(f"Error: {e}")
            results['error_count'] += 1
    
    return results

# Test all models
all_results = []
for name, model_data in loaded_models.items():
    print(f"Testing {name}...")
    result = test_algorithm_on_dataset(name, model_data, test_df)
    if result['success_count'] > 0:
        all_results.append(result)
        print(f"✅ {name}: {result['success_count']} predictions")

In [ ]:
# Ensemble methods (soft voting, hard voting, etc.)
def soft_voting(results):
    """Soft voting ensemble"""
    n_samples = len(results[0]['predictions'])
    n_classes = len(EMOTION_CLASSES)
    
    prob_sum = np.zeros((n_samples, n_classes))
    
    for r in results:
        for i, (pred, conf) in enumerate(zip(r['predictions'], r['confidences'])):
            prob_sum[i, pred] += conf
    
    prob_sum /= len(results)
    predictions = np.argmax(prob_sum, axis=1)
    confidences = np.max(prob_sum, axis=1)
    
    return predictions, confidences

# Apply ensemble
if len(all_results) > 1:
    ensemble_preds, ensemble_confs = soft_voting(all_results)
    
    ensemble_result = {
        'algorithm': 'Soft_Voting_Ensemble',
        'predictions': ensemble_preds.tolist(),
        'ground_truths': all_results[0]['ground_truths'],
        'confidences': ensemble_confs.tolist(),
        'success_count': len(ensemble_preds),
        'error_count': 0
    }
    
    all_results.append(ensemble_result)
    print("✅ Ensemble methods applied")

In [ ]:
# Calculate metrics
performance_data = []

for result in all_results:
    if result['success_count'] > 0:
        acc = accuracy_score(result['ground_truths'], result['predictions'])
        precision, recall, f1, _ = precision_recall_fscore_support(
            result['ground_truths'], 
            result['predictions'], 
            average='weighted', 
            zero_division=0
        )
        
        performance_data.append({
            'Algorithm': result['algorithm'],
            'Accuracy': acc,
            'Precision': precision,
            'Recall': recall,
            'F1_Score': f1,
            'Avg_Confidence': np.mean(result['confidences'])
        })

performance_df = pd.DataFrame(performance_data)
performance_df = performance_df.sort_values('Accuracy', ascending=False)

print("\n🏆 FINAL LEADERBOARD:")
print(performance_df)

# Visualization
plt.figure(figsize=(12, 6))
plt.bar(performance_df['Algorithm'], performance_df['Accuracy'], color='skyblue', edgecolor='navy')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison (3-Class System)')
plt.grid(axis='y', alpha=0.3)

for i, (algo, acc) in enumerate(zip(performance_df['Algorithm'], performance_df['Accuracy'])):
    plt.text(i, acc + 0.005, f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Confusion matrix for best model
best_result = all_results[0]  # Assuming sorted by accuracy
cm = confusion_matrix(best_result['ground_truths'], best_result['predictions'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=EMOTION_CLASSES, 
            yticklabels=EMOTION_CLASSES)
plt.title(f"Confusion Matrix - {best_result['algorithm']}")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Save results
performance_df.to_csv('3class_performance_results.csv', index=False)
print("✅ Results saved to 3class_performance_results.csv")

## Key Differences from Your Original:

1. **No class merging** - Your system is already 3-class (angry, happy, relaxed)
2. **Correct branch** - Using `conf-3cls` instead of `conf-merge-3cls`
3. **Simplified YOLO handling** - No conversion needed if YOLO outputs match
4. **Cleaner model loading** - Proper parameter passing
5. **Removed unnecessary conversion functions**

## Notes:

- Ensure your YOLO model outputs match the 3-class system
- Verify model file paths are correct
- The notebook assumes models are trained on the same 3-class configuration
- If any model was trained on 4-class, you'll need conversion logic

## Expected Output:

The notebook will:
1. ✅ Download and load all models
2. ✅ Process test dataset (no class conversion)
3. ✅ Test each model individually
4. ✅ Apply ensemble methods
5. ✅ Generate performance metrics and visualizations
6. ✅ Save results to CSV

**Final Result:** Performance comparison across all algorithms with 3-class emotion recognition (angry, happy, relaxed)